In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
country     = pd.read_csv("country.csv")
league      = pd.read_csv("league.csv")
match       = pd.read_csv("match.csv")
match['date']  = pd.to_datetime(match['date'])
match = match[match.isnull().sum(axis=1) < 23]
ptemp       = pd.read_csv("player_attributes.csv") 
ptemp = ptemp.drop_duplicates(subset=['player_id', 'date'])
ptemp['date']  = pd.to_datetime(ptemp['date']) # add ending date/time for which player info is valid
player_att  = ptemp.assign(next_date=ptemp.groupby('player_id').date.shift(1))
player_att.to_csv("player_att__with_endtime.csv")
player      = pd.read_csv("player.csv")
ttemp       = pd.read_csv("team_attributes.csv")  
ttemp['date']  = pd.to_datetime(ttemp['date'])  # add ending date/time for which team info is valid
team_att    = ttemp.assign(next_date=ttemp.groupby('team_id').date.shift(-1))
team_att.to_csv("team_att_with_endtime.csv")
team        = pd.read_csv("team.csv")

In [3]:
t_att_lab = [att for att in team_att.columns[2:-1]]
ht_lab = ['h_' + att for att in t_att_lab]
at_lab = ['a_' + att for att in t_att_lab]
p_att_lab = [att for att in player_att.loc[:,'overall_rating':'next_date'].select_dtypes("float64")]
p_att_list = ['h_' + att for att in p_att_lab]
p_att_list.extend(['a_' + att for att in p_att_lab])
for att in ht_lab:
    match[att] = ''
for att in at_lab:
    match[att] = ''
for att in p_att_list:
    match[att] = ''

In [4]:
def calc_player_info(row):
    date = row['date']
    h_players = [player for player in row.iloc[10:21] if player == player]
    h = player_att[player_att['player_id'].isin(h_players)]
    h = h[(h['date'] < date) & ((h['next_date'] != h['next_date']) | (h['next_date'] > date))][p_att_lab]
    h = h.add_prefix('h_')
    h_out = h.mean(axis=0)

    a_players = [player for player in row.iloc[21:32] if player == player]
    a = player_att[player_att['player_id'].isin(a_players)]
    a = a[(a['date'] < date) & ((a['next_date'] != a['next_date']) | (a['next_date'] > date))][p_att_lab]
    a = a.add_prefix('a_')
    a_out = a.mean(axis=0)
    return h_out.append(a_out)

def calc_h_team(row):
    date = row['date']
    hteam = team_att[team_att['team_id'] == row['home_team_id']]
    hteam = hteam[(hteam['date'] <= date) & ((hteam['next_date'] != hteam['next_date']) | (hteam['next_date'] > date))][t_att_lab]
    hteam.columns = ['h_' + col for col in hteam.columns]
    if hteam.shape[0] > 0:
        return hteam.iloc[0].squeeze()
    else: 
        return [np.nan] * len(t_att_lab)

def calc_a_team(row):
    date = row['date']
    ateam = team_att[team_att['team_id'] == row['away_team_id']]
    ateam = ateam[(ateam['date'] < date) & ((ateam['next_date'] != ateam['next_date']) | (ateam['next_date'] > date))][t_att_lab]
    ateam.columns = ['a_' + col for col in ateam.columns]
    if ateam.shape[0] > 0:
        return ateam.iloc[0].squeeze()
    else:
        return [np.nan] * len(t_att_lab)

In [5]:
match[ht_lab] = match.apply(calc_h_team, axis=1, result_type='expand')
print("Home Team labels applied.")
match[at_lab] = match.apply(calc_a_team, axis=1, result_type='expand')
print("Away Team labels applied.")
match[p_att_list] = match.apply(calc_player_info, axis=1)
print("Player labels applied.")
match.to_csv(f"match_cleaned.csv")
match = match[match.isnull().sum(axis=1) < 30]
match.to_csv(f"match_cleaned_drop30.csv")

Home Team labels applied.
Away Team labels applied.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
18000
19000
20000
21000
22000
23000
24000
Player labels applied.
